In [2]:
import csv
import pickle
import pandas as pd
import numpy as np
from typing import List, Tuple, Generator
from tqdm import tqdm

In [14]:
# Load the pickle file
with open('training-data-bitstrings-1727884000295915700.pkl', 'rb') as f:
    data = pickle.load(f)

# Unpack the data
X, y = data

In [15]:
def analyze_raw_pickle_data(file_path: str, num_samples: int = 5):
    # Load the pickle file
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    
    # Unpack the data
    X, y = data
    
    print(f"Total number of samples: {len(X)}")
    print(f"Shape of y: {np.array(y).shape}")
    
    print(f"\nExamining the first {num_samples} samples:")
    for i in range(min(num_samples, len(X))):
        print(f"\nSample {i+1}:")
        print(f"Number of filters: {len(X[i])}")
        
        for j, filter_bytes in enumerate(X[i]):
            binary = ''.join(format(byte, '08b') for byte in filter_bytes)
            print(f"  Filter {j+1}: {len(binary)} bits")
            print(f"    First 100 bits: {binary[:100]}...")
            print(f"    Last 100 bits: ...{binary[-100:]}")
        
        print(f"y value: {y[i]}")
    
    return X, y

In [16]:
X, y = analyze_raw_pickle_data('training-data-bitstrings-1727884000295915700.pkl')

Total number of samples: 10000
Shape of y: (10000, 2)

Examining the first 5 samples:

Sample 1:
Number of filters: 1
  Filter 1: 12848 bits
    First 100 bits: 0000011100000000000000000000000000000000000000000000000000000000101001100001111010101010110110111110...
    Last 100 bits: ...0000001001001000000100101010100110010001100110110011010010000100010100000000111011100111000001010001
y value: [774. 169.]

Sample 2:
Number of filters: 2
  Filter 1: 12848 bits
    First 100 bits: 0000011100000000000000000000000000000000000000000000000000000000101100101010000001010011111010010101...
    Last 100 bits: ...0100000101010010000110100011101100011000001001011100000110001010100111000011110100010110110010111010
  Filter 2: 88 bits
    First 100 bits: 0000011100000000000000000000000000000000000000000000000000000000000000100100001011001001...
    Last 100 bits: ...0000011100000000000000000000000000000000000000000000000000000000000000100100001011001001
y value: [967. 963.]

Sample 3:
Number of filt

In [17]:
def create_csv_from_standardized(standardized_generator, y_target, df_y, output_file='processed_data_small.csv', batch_size=1000):
    total_samples = len(y_target)
    
    with open(output_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write header
        csvwriter.writerow(['concatenated_bitstrings', 'num_included', 'num_excluded'])
        
        sample_index = 0
        for batch in tqdm(standardized_generator, desc="Processing batches", total=(total_samples + batch_size - 1) // batch_size):
            for i in range(len(batch)):
                if sample_index >= total_samples:
                    break
                
                # Concatenate all filters for this sample
                concatenated_bitstring = ''.join(batch[i])
                
                # Get the corresponding num_included and num_excluded
                num_included = y_target[sample_index]
                num_excluded = df_y['num_excluded'].values[sample_index]
                
                # Write the row
                csvwriter.writerow([concatenated_bitstring, num_included, num_excluded])
                
                sample_index += 1
            
            if sample_index >= total_samples:
                break

def calculate_max_filters_and_lengths(X: List[List[bytes]]) -> Tuple[int, List[int]]:
    print("Calculating maximum filters and lengths...")
    max_filters = max(len(sample) for sample in X)
    max_lengths = [0] * max_filters
    for sample in tqdm(X, desc="Samples processed"):
        for i, filter_bytes in enumerate(sample):
            if i < max_filters:
                max_lengths[i] = max(max_lengths[i], len(''.join(format(byte, '08b') for byte in filter_bytes)))
    return max_filters, max_lengths

def standardize_dataset(X: List[List[bytes]], max_filters: int, max_lengths: List[int], batch_size: int = 1000) -> Generator[np.ndarray, None, None]:
    def bytes_to_binary(byte_string: bytes) -> str:
        return ''.join(format(byte, '08b') for byte in byte_string)

    print("Standardizing dataset...")
    total_batches = (len(X) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(X), batch_size), total=total_batches, desc="Batches processed"):
        batch = X[i:i+batch_size]
        standardized_batch = []
        for sample in batch:
            standardized_sample = []
            for j in range(max_filters):
                if j < len(sample):
                    binary_str = bytes_to_binary(sample[j])
                    filter_str = binary_str.ljust(max_lengths[j], '0')[:max_lengths[j]]
                else:
                    filter_str = '0' * max_lengths[j]
                standardized_sample.append(filter_str)
            standardized_batch.append(standardized_sample)
        
        yield np.array(standardized_batch)

def process_large_dataset(X: List[List[bytes]], y, df_y, max_filters: int = None, batch_size: int = 1000, output_file: str = 'processed_data.csv'):
    try:
        # Step 1: Calculate max_filters and max_lengths
        if max_filters is None:
            max_filters, max_lengths = calculate_max_filters_and_lengths(X)
        else:
            _, max_lengths = calculate_max_filters_and_lengths(X)
        
        print(f"Max number of filters: {max_filters}")
        print(f"Max lengths of filters: {max_lengths}")

        # Step 2: Process the dataset and create CSV
        print("Starting dataset standardization and CSV creation...")
        standardized_generator = standardize_dataset(X, max_filters, max_lengths, batch_size)
        
        data = []  # List to store all processed data
        
        with open(output_file, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['concatenated_bitstrings', 'num_included', 'num_excluded'])
            
            batch_count = 0
            sample_index = 0
            for batch in standardized_generator:
                batch_count += 1
                
                if batch_count == 1:
                    print(f"Shape of standardized batch: {batch.shape}")
                    # Display a few samples from the first batch to verify
                    for i in range(min(3, len(batch))):
                        print(f"\nSample {i+1}:")
                        for j, filter_str in enumerate(batch[i]):
                            print(f"  Filter {j+1}: {len(filter_str)} bits")
                            print(f"    First 100 bits: {filter_str[:100]}...")
                
                for i in range(len(batch)):
                    if sample_index >= len(X):
                        break
                    
                    concatenated_bitstring = ''.join(batch[i])
                    num_included = y[sample_index][0]  # Take the first element of the list
                    num_excluded = y[sample_index][1]  # Take the second element of the list
                    
                    row = [concatenated_bitstring, num_included, num_excluded]
                    csvwriter.writerow(row)
                    data.append(row)
                    sample_index += 1
                
                print(f"Processed batch {batch_count}, shape: {batch.shape}")

        print(f"Total number of batches processed: {batch_count}")
        print(f"Dataset standardization and CSV creation completed. Output saved to {output_file}")

        # Create DataFrame from processed data
        df = pd.DataFrame(data, columns=['concatenated_bitstrings', 'num_included', 'num_excluded'])
        
        return df
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("You can retry the process from where it left off or debug the specific step.")
        return None

In [18]:
df_y = pd.DataFrame(y, columns=['num_included', 'num_excluded'])

df_y.head()

,num_included,num_excluded
0,774.0,169.0
1,967.0,963.0
2,980.0,3.0
3,296.0,24.0
4,279.0,930.0


In [19]:
df = process_large_dataset(X, y, df_y, max_filters=None, batch_size=1000, output_file='processed_data.csv')


Calculating maximum filters and lengths...


Samples processed: 100%|██████████| 10000/10000 [00:05<00:00, 1828.23it/s]


Max number of filters: 5
Max lengths of filters: [12848, 192, 216, 112, 88]
Starting dataset standardization and CSV creation...
Standardizing dataset...


Batches processed:   0%|          | 0/10 [00:00<?, ?it/s]

Shape of standardized batch: (1000, 5)

Sample 1:
  Filter 1: 12848 bits
    First 100 bits: 0000011100000000000000000000000000000000000000000000000000000000101001100001111010101010110110111110...
  Filter 2: 192 bits
    First 100 bits: 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000...
  Filter 3: 216 bits
    First 100 bits: 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000...
  Filter 4: 112 bits
    First 100 bits: 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000...
  Filter 5: 88 bits
    First 100 bits: 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000...

Sample 2:
  Filter 1: 12848 bits
    First 100 bits: 0000011100000000000000000000000000000000000000000000000000000000101100101010000001010011111010010101...
  Filter 2: 192 bits
    First 100 bits: 0000011100000000000000000000000000000

...

Processed batch 8, shape: (1000, 5)


Batches processed:  90%|█████████ | 9/10 [00:09<00:01,  1.02s/it]

Processed batch 9, shape: (1000, 5)


Batches processed: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]

Processed batch 10, shape: (1000, 5)
Total number of batches processed: 10
Dataset standardization and CSV creation completed. Output saved to processed_data.csv


In [20]:
df.head()

concatenated_bitstrings  \
0  0000011100000000000000000000000000000000000000000000000000000000101001100001111010101010110110111110110110000100000010100000110110001010100100011001000001011101010010011100001010000000110001010001111100001001100000111101000101000011101110111100100010010110101011111001010110010101001011010001110001101011111101011101000110011111010111110100011100101100010000110001110010110101000110110111000000000101100011010001010000110011001110000010101010001000100110010111001100100111001101110010100010100111110000100001000001010100101000100011000000100101001111000001101110101000100101000001011010101111000110111111000000000000111100100100000010100101101001011100001000001101001110111011101100000010100110101000000000010100110111101011001110000011111001011100001110110111100101111110011010000010111000010100000111001100100011000000001101110011100100001000011001110011001100000100100101101010010001000100101101110000001100100010000001000011011011110101100011011000001010011110001000011100110000111101100010010000101001000011110110000100101000101110001100011101100110011111000101101100110001000111010000011100100101100100011101001001000010000111001011110001000101101010001010111100101101000000000000010011001000111001111001100010111000101000001010011010010000000001010110100000101000110010001010011010101110000010111001110111100110111111111110101011000000110100111001011001111101111010101110011111000001111000001110100010100010010010100011101101000101010110001010101000001100001000011011001001100110001101000001001110001000100110000100001011101100001110011100110000001111000010000100010111000101001110000110011110111000001100100010010001000100000111101101111011101101100010000101000000101100011000001101010000001000100001100000111011000110001000010101010010000100100110101111111011100000110111010110100110110101101010001000011100110110001111110000010000000010110000100110011001010001010001011100001000111001111010000000000010110100110001110001100101101010100010100111000100001010101100010110101010010010100001000010001100001011110101000010011100100001110100000110010100000111000000111000111000001001100000100101110110001110001101011111000000100101000000010101000010010000101000000110001100111000110100110110011101000100100100010011100011001001000100101010111000001010001010110000000000101100011010011000011110110000000110000010001100000000110000010100010101011010110100001011000100100011000000010001010111101110111100110010000010000000100011100000011011010111000110110101011111101000011111101101000101001011000000000000000010010110000111110010011011000110110001010111110110000000010010110001011110000111011100000011010111101011011011010110101000101010001010001010100000000011001101101011011101010101010001101000110000010001111000010010001110010110011000011110001110010111100000110000000100000000001100001001001001010000010000011110100110100100011100010000101100101001011011111100110001001100101101001101010011011001101100000001000001001100101100010110000100111110101111110000110010101001001011000011000001101100001100110010000010101101010100111001101010010001100000100111000001000110011011110110111101110111000110001100010000001001011110010000010111010100000110111000010000000111000110111101110111001011000000010101101000000100000000011111010010100111100001100100110001110001100100011110001010101001010100010111010001110010100100010110001000010000000000001100001001011000000110001110000101000111011110100100010101101101000001001001000011001010001111110000001110100010100010100001000010101101001011110101100000001011000001000011000001101100011101110010101011111010111110110100000110010010000011011000001001001001110101000000100000001000001111011101000001000001001011110110000101110000101101010000101111100100000101011110110011000100000000100011010010110001111100010100001101000111011010111010110001011001001000101010010000001100010011000111010000100001101110000000001110011100100111001001000000001111001010011000000100000110111010001010000000101110000111010011111101110110011000000110

In [21]:
pd.set_option('display.max_colwidth', None)  # This will display the full content of each cell

def analyze_processed_data(df, num_samples=5):
    # Get basic information about the dataset
    total_rows = len(df)
    num_columns = len(df.columns)
    
    print(f"Total number of rows: {total_rows}")
    print(f"Number of columns: {num_columns}")
    print("\nColumn names:")
    print(df.columns.tolist())
    
    print("\nData types of columns:")
    print(df.dtypes)
    
    print(f"\nFirst {num_samples} rows of the dataset:")
    for i, row in df.head(num_samples).iterrows():
        print(f"\nRow {i+1}:")
        print(f"concatenated_bitstrings (length {len(row['concatenated_bitstrings'])}):")
        print(row['concatenated_bitstrings'])
        print(f"num_included: {row['num_included']}")
        print(f"num_excluded: {row['num_excluded']}")
    
    print("\nSummary statistics for numeric columns:")
    print(df.describe())
    
    # Additional information about the 'concatenated_bitstrings' column
    bitstring_length = len(df['concatenated_bitstrings'].iloc[0])
    print(f"\nLength of each concatenated bitstring: {bitstring_length}")

    return df

In [22]:
df = analyze_processed_data(df)

Total number of rows: 10000
Number of columns: 3

Column names:
['concatenated_bitstrings', 'num_included', 'num_excluded']

Data types of columns:
concatenated_bitstrings     object
num_included               float64
num_excluded               float64
dtype: object

First 5 rows of the dataset:

Row 1:
concatenated_bitstrings (length 13456):
0000011100000000000000000000000000000000000000000000000000000000101001100001111010101010110110111110110110000100000010100000110110001010100100011001000001011101010010011100001010000000110001010001111100001001100000111101000101000011101110111100100010010110101011111001010110010101001011010001110001101011111101011101000110011111010111110100011100101100010000110001110010110101000110110111000000000101100011010001010000110011001110000010101010001000100110010111001100100111001101110010100010100111110000100001000001010100101000100011000000100101001111000001101110101000100101000001011010101111000110111111000000000000111100100100000010100101101001011100001